In [1]:
!pip install pyngrok
!ngrok authtoken # write down your auth token here #

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [2]:
from PIL import Image
from flask import Flask, request
from pyngrok import ngrok

import torch
import numpy as np
import torchvision.transforms as transforms
from skimage.io import imread
from skimage.transform import resize as imresize

from google.colab import drive
drive.mount('/content/drive')

In [3]:
def image_preprocess(filepath):
  img = np.array(Image.open(filepath))
  if len(img.shape) == 2:
      img = img[:, :, np.newaxis]
      img = np.concatenate([img, img, img], axis=2)
  img = imresize(img, (256, 256)) # constant', 'edge', 'symmetric', 'reflect', 'wrap'
  img = img.transpose(2, 0, 1)
  # img = img / 255.
  img = torch.FloatTensor(img) # .to(device)
  normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
  transform = transforms.Compose([normalize])
  imagexx = transform(img)  # (3, 256, 256)

  return imagexx.unsqueeze(0)

In [4]:
model = torch.jit.load('drive/MyDrive/caption_model/test_ic111.pt')
_ = model.eval()


In [6]:
app = Flask(__name__)
public_url = ngrok.connect("5000", domain = "expert-smoothly-insect.ngrok-free.app")
app.config["BASE_URL"] = public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\ ")

@app.route("/imageUpload", methods = ['POST'])
def processImage():
    f = request.files['image']
    return {"answer": model(image_preprocess(f), 3)}

app.run()

 * ngrok tunnel "NgrokTunnel: "https://expert-smoothly-insect.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000\ 
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Jun/2024 04:40:14] "POST /imageUpload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Jun/2024 04:40:24] "POST /imageUpload HTTP/1.1" 200 -


In [ ]:
ngrok.get_tunnels()

[<NgrokTunnel: "https://expert-smoothly-insect.ngrok-free.app" -> "http://localhost:5000">]

In [ ]:
ngrok.kill()